# Bayesian Experimental Design

This demo shows how to use the GPUncertaintyOptimizer class to efficiently obtain new samples that will decrease the predictive uncertainty of a GP model.

As a test case, we use the GaussianProcessRegressor class to calculate the efficiency of three-jet events with MET < 50 GeV.

The following nuisance parameters are considered:
- $\nu_{J1in}$: Jet energy scale of the leading jet, $J_1$, when |$\eta_1$| < 1.
- $\nu_{J1out}$: Jet energy scale of the leading jet, $J_1$, when |$\eta_1$| >= 1.
- $\nu_{J23in}$: Jet energy scale of the two softer jets, $J_2$ and $J_3$, when |average($\eta_2$, $\eta_3$)| < 1.
- $\nu_{J23out}$: Jet energy scale of the two softer jets, $J_2$ and $J_3$, when |average($\eta_2$, $\eta_3$)| >= 1.

In [2]:
# Imports
import gpder
from gpder.gaussian_process import GaussianProcessRegressor
from gpder.gaussian_process.kernels import RegularKernel, DerivativeKernel
from gpder.bayes import GPUncertaintyOptimizer

import numpy as np 
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Rectangle, Arrow, FancyArrow
%matplotlib inline  

from multiprocessing import Pool, cpu_count

In [3]:
from utils import download_dataset, load_dataset

# Downloading the dataset from zenodo. 
# By default, the h5df file is saved in the current directory.
download_dataset()
threeM = load_dataset()

# The dataset consists of 30000 events, each with three jets. 
# For each jet, the three-momenta (pT, eta, phi) are saved in descending pT-order.
print("Shape of the dataset:", threeM.shape)

Shape of the dataset: (30000, 3, 3)


Title: Estimation of Unfactorizable Systematic Uncertainties
Keywords: 
Publication date: 2024-04-14
DOI: 10.5281/zenodo.10971439
Total size: 1.0 MB

Link: https://zenodo.org/api/records/10971439/files/three_jets_30k.h5/content   size: 1.0 MB
three_jets_30k.h5 is already downloaded correctly.
All files have been downloaded.


In [4]:
# Function that calculates the efficiency with respect to the two nuisance
# See hep_functions.py for more details
from hep_functions import efficiency
def efficiency_events(X, threeM=threeM):
    # Simplifying the input
    return efficiency(X, threeM)


# And the function that calculates the gradient of the efficiency
# To improve numerical stability, we smooth out the gradients by setting the
# parameter a=1/10
from hep_functions import der_efficiency
def der_efficiency_events_sigmoid10(X, threeM=threeM):
    # Simplifying the input
    return der_efficiency(X, threeM, a=1/10)

## Regular GP regression

The regular GP used in the regular_GP_regression notebook has nine training samples. Here, we use the BED strategy to select 30 new training samples. 

In [5]:
# -- test dataset ---------------------------------------------------------- #
res_test = 25
X_lower, X_upper = 0.5, 1.5
lin = np.linspace(X_lower, X_upper, res_test)
nu_J1_in_test, nu_J1_out_test, nu_J23_in_test, nu_J23_out_test = np.meshgrid(
    lin, lin, lin, lin
)
X_test = np.array(
    [
        nu_J1_in_test.flatten(),
        nu_J1_out_test.flatten(),
        nu_J23_in_test.flatten(),
        nu_J23_out_test.flatten(),
    ]
).T
y_test = Pool(cpu_count()).map(efficiency_events, X_test)
y_test = np.array(y_test)
# -------------------------------------------------------------------------- #

In [6]:
def generate_training_grid(res):
    cent_val = (X_lower + X_upper) / 2.0
    st = (cent_val - X_lower) / 2.0
    X_coords = np.linspace(np.repeat(X_lower, 4) + st, np.repeat(X_upper, 4) - st, res)
    n = len(X_coords)
    X_grid = np.ones((n * 4, 4)) * cent_val
    for i in range(4):
        X_grid[i * n : (i + 1) * n, i] = X_coords[:, i]
    X_grid_set = set()
    for x in X_grid:
        tupl = tuple(x)
        if tupl not in X_grid_set:
            X_grid_set.add(tupl)
    return np.array(list(X_grid_set))


# -- trainning dataset ----------------------------------------------------- #
X_train = generate_training_grid(res=3)
y_train = Pool(cpu_count()).map(efficiency_events, X_train)
y_train = np.array(y_train).reshape(
    -1,
)
dX_train = X_train
dy_train = Pool(cpu_count()).map(der_efficiency_events_sigmoid10, X_train)
dy_train = np.array(dy_train).reshape(-1, 4)
# -------------------------------------------------------------------------- #

In [7]:
# -- utility input --------------------------------------------------------- #
# The utility input is used to evaluate the net predictive variance of the
# GP model at every BED iteration.
lin = np.linspace(X_lower, X_upper, 5)
nuJ1_in, nuJ1_out, nuJ23_in, nuJ23_out = np.meshgrid(lin, lin, lin, lin)
X_util = np.vstack(
    (nuJ1_in.flatten(), nuJ1_out.flatten(), nuJ23_in.flatten(), nuJ23_out.flatten())
).T
# -------------------------------------------------------------------------- #

In [10]:
# -- fitting the GP model and hyperparameter optimization ------------------ #
kernel = RegularKernel(amplitude=0.1, length_scale=0.25, noise_level=1e-3)
# set optimizer=None to skip optimization
gp_reg = GaussianProcessRegressor(kernel=kernel, optimizer=None, random_state=42)
gp_reg.fit(X_train, y_train)
print(gp_reg.kernel)
# -------------------------------------------------------------------------- #

# -- BED ------------------------------------------------------------------- #
BED_reg = GPUncertaintyOptimizer(
    gp_model=gp_reg,
    bounds={
        "nu_J1_in": (X_lower, X_upper),
        "nu_J1_out": (X_lower, X_upper),
        "nu_J23_in": (X_lower, X_upper),
        "nu_J23_out": (X_lower, X_upper),
    },
    function=efficiency_events,
    random_state=42,
    verbose=True,
)
gp_reg = BED_reg.minimize_variance(
    X_util=X_util, n_iters=30, n_restarts_optimizer=10, added_noise=None
)
# -------------------------------------------------------------------------- #

0.1**2 * RBF(length_scale=0.25) + WhiteKernel(noise_level=0.001)
| Iter |  nu_J1_in  | nu_J1_out  | nu_J23_in  | nu_J23_out |   Target   |
-------------------------------------------------------------------------
|  0   |    1.25    |    1.00    |    1.00    |    1.00    |    0.96    |
|  0   |    1.00    |    1.00    |    1.25    |    1.00    |    0.61    |
|  0   |    1.00    |    1.00    |    0.75    |    1.00    |    0.91    |
|  0   |    1.00    |    1.00    |    1.00    |    0.75    |    0.88    |
|  0   |    1.00    |    1.25    |    1.00    |    1.00    |    0.98    |
|  0   |    0.75    |    1.00    |    1.00    |    1.00    |    0.52    |
|  0   |    1.00    |    0.75    |    1.00    |    1.00    |    0.46    |
|  0   |    1.00    |    1.00    |    1.00    |    1.00    |    1.00    |
|  0   |    1.00    |    1.00    |    1.00    |    1.25    |    0.64    |
|  1   |    0.67    |    0.67    |    0.67    |    1.33    |    0.07    |
|  2   |    1.33    |    1.33    |    1.33    |

## Derivative GP regression

We also use the BED strategy to select 30 new training samples for the derivative GP model from the derivative_GP_regression notebook. 

In [12]:
# -- trainning dataset ----------------------------------------------------- #
dy_train = Pool(cpu_count()).map(der_efficiency_events_sigmoid10, X_train)
dy_train = np.array(dy_train)
# -------------------------------------------------------------------------- #

In [14]:
# -- fitting the GP model and hyperparameter optimization ------------------ #
kernel = DerivativeKernel(
    amplitude=0.1, length_scale=0.25, noise_level=1e-3, noise_level_der=1
)
# set optimizer=None to skip optimization
gp_der = GaussianProcessRegressor(kernel=kernel, optimizer=None, random_state=42)
gp_der.fit(X_train, y_train, X_train, dy_train)
print(gp_der.kernel)
# -------------------------------------------------------------------------- #

# -- BED ------------------------------------------------------------------- #
BED_der = GPUncertaintyOptimizer(
    gp_model=gp_der,
    bounds={
        "nu_J1_in": (X_lower, X_upper),
        "nu_J1_out": (X_lower, X_upper),
        "nu_J23_in": (X_lower, X_upper),
        "nu_J23_out": (X_lower, X_upper),
    },
    function=efficiency_events,
    der_function=der_efficiency_events_sigmoid10,
    random_state=42,
    verbose=True,
)
gp_der = BED_der.minimize_variance(
    X_util=X_util, n_iters=30, n_restarts_optimizer=10, added_noise=None
)
# -------------------------------------------------------------------------- #

0.1**2 * DerivativeRBF(length_scale=0.25) + WhiteKernel(noise_level=0.001) + WhiteKernel_der(noise_level=1)
| Iter |  nu_J1_in  | nu_J1_out  | nu_J23_in  | nu_J23_out |   Target   |
-------------------------------------------------------------------------
|  0   |    1.25    |    1.00    |    1.00    |    1.00    |    0.96    |
|  0   |    1.00    |    1.00    |    1.25    |    1.00    |    0.61    |
|  0   |    1.00    |    1.00    |    0.75    |    1.00    |    0.91    |
|  0   |    1.00    |    1.00    |    1.00    |    0.75    |    0.88    |
|  0   |    1.00    |    1.25    |    1.00    |    1.00    |    0.98    |
|  0   |    0.75    |    1.00    |    1.00    |    1.00    |    0.52    |
|  0   |    1.00    |    0.75    |    1.00    |    1.00    |    0.46    |
|  0   |    1.00    |    1.00    |    1.00    |    1.00    |    1.00    |
|  0   |    1.00    |    1.00    |    1.00    |    1.25    |    0.64    |
|  1   |    0.67    |    0.67    |    0.67    |    1.33    |    0.07    |
|  2